In [1]:
from sage.all import *
import sys
import hashlib
import random
import math


In [2]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id
gmsk = load(data_path + 'sk' + key_suffix)
pk = load(data_path + 'pk')

In [3]:
n = 2048
l = 170
sigma_1 = 0.16
U = pk[0]
B = pk[1]
A = pk[2]

S_A = gmsk[0]
P_A = gmsk[1]
a_0 = gmsk[2]
a = gmsk[3]
g1 = gmsk[4]
#     show(U)
#     show(B)
#     show(A)
#     show(g1)

F = a.parent()
PR = g1.parent()
X = PR.gen()
K_ = a_0.parent()
L = []
L.append(F(0))
for i in range(2047):
    L.append(F(a**i))
# L = [F(a**i) for i in range(2048)]
# SyndromeCalculator = matrix(PR, 1, len(L));
# for i in range(len(L)):
#     SyndromeCalculator[0, i] = (X - L[i]).inverse_mod(g1);

# h = PR(1)
# for a_i in L:
#     h = h * (X - a_i);
# gamma = [];

# for a_i in L:
#     gamma.append((h * ((X - a_i).inverse_mod(g1))).mod(g1));

# t = 32    
# H_check_poly = matrix(F, t, n);
# for i in range(n):
#     coeffs = list(gamma[i]);
#     for j in range(t):
#         # Check to make sure the coefficient exists
#         # (It may not if the polynomial is not of
#         # degree t)
#         if j < len(coeffs):
#             H_check_poly[j, i] = coeffs[j];
#         else:
#             H_check_poly[j, i] = F(0);

# # Construct the binary parity-check matrix for the Goppa code.
# # Do so by converting each element of F_2**m to its binary
# # representation.
# H_Goppa = matrix(K_, 11 * H_check_poly.nrows(), H_check_poly.ncols());
# for i in range(H_check_poly.nrows()):
#     for j in range(H_check_poly.ncols()):
#         be = bin(eval(H_check_poly[i, j]._int_repr()))[2:];
# #             be = bin(eval(H[i,j]._int_repr()))[2:];
#         be = '0'*(11 - len(be)) + be;
#         be = list(be);
#         H_Goppa[11 * i:11 * (i + 1), j] = vector(map(int, be));
        
# G_Goppa = H_Goppa.right_kernel().basis_matrix();

In [4]:
def gen_random_fix_vec(n, w):
    vec = [1]*w+(n-w)*[0]
    random.shuffle(vec)
    return vec

e=gen_random_fix_vec(n,16)
e=vector(GF(Integer(2)),e)

In [5]:
def get_defining_set(n, pol, field):
    """
    Return a tuple of ``n`` distinct elements of ``field``
    that are not roots of ``pol`` 
    
    INPUT:
        - ``n`` -- length
        - ``pol`` -- a monic polynomial with coefficients in `field`
        - ``field`` -- finite field
    """
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set


In [6]:
def random_word(n, field):
    """
    Return a random vector of length ``n`` belonging to the 
    finite field ``field``
    
    INPUT:
        - ``n`` -- length
        - ``field``-- finite field
    """
    v = []

    while len(v) != n:
        v = v + [choice(field.list())]

    word = vector(field, v)
    
    return word

In [7]:
def random_error(n, num_errors, field):
    """
    Return a random vector of length ``n`` with maximum weight 
    ``num_errors`` belonging to the finite field ``field``
    
    INPUT:
        - ``n`` -- length
        - ``num_errors`` -- maximum weight
        - ``field`` -- finite field
    """
    v = [0] * n
    i = 0
    
    while i < num_errors:
        m = randint(0, len(v)-1)
        v[m] = choice(field.list())
        i += 1

    e = vector(field, v)
    
    return e

In [8]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`
    - ``defining_set`` -- tuple of `n` distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if not self._field.is_field() or not self._field.is_finite():
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
        if self.get_dimension() == 0:
            raise ValueError("ERROR. Code dimension is null")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())

In [9]:
class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        self._generator_matrix = self.code().get_generator_matrix()
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self._generator_matrix
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m`` -- a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

In [10]:
class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c`` -- a element of the input space of ``self``
        """   
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):
            syndrome = syndrome + c[i]*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword
        
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 0

        # Step 1
        _S = self.get_syndrome(word)
        
        if _S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = _S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t or r_prev.degree() < t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
        
        # Step 5
        # make sigma monic     
        sigma = U_i / U_i.coefficients()[-1]
        
        eta = (-1)**i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        for root in sigma.roots():
            roots_loc = roots_loc + [self._defining_set.index(root[0])]
#             if C.get_generating_pol()(root[0]) != 0:
#                 roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(roots_loc)):
#             error[roots_loc[i]] = 1
             if (eta.subs(sigma.roots()[i][0]) != 0):
                 error[roots_loc[i]] = 1
#             error[roots_loc[i]] = eta.subs(sigma.roots()[i][0]) / (sigma_diff.subs(sigma.roots()[i][0]))
#             error[roots_loc[i]] = eta.subs(x = sigma.roots()[i][0]) / (sigma_diff.subs(x = sigma.roots()[i][0]))
        
        x = word - vector(self.code()._field, error)
        
        return x

    def decode_to_message(self, word):
        r"""
        Decode ``word`` to the message space.
        
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        word = self.decode_to_code(word)
        message = self.code().get_generator_matrix().solve_left(word)
        return message

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [11]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic
# defining_set = get_defining_set(n, g1, L)

In [12]:
# Goppa code
C = Goppa(L, g1, K_)

print("Goppa code:")
print(C)

print("\nParity matrix:")
H = C.get_parity_check_matrix()
show(H)
#print(latex(H))

print("\nGenerator matrix:")
G = C.get_generator_matrix()
show(G)

Goppa code:
[2048, 1696] Goppa code

Parity matrix:
352 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

Generator matrix:
1696 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [13]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nWord to encode:")
word = random_word(G.nrows(), K_)
print(word)

print("\nWord encoded:")
x = E.encode(word)
print("x = " + str(x))

print("\nErrors added to x:")
num_errors = floor(g1.degree()/2)
e = random_error(len(x), num_errors, K_)
print("e = " + str(e))

print("\nWord encoded with errors (y = x + e):")
y = x + e
print("y = " + str(y))


Goppa encoder:
Encoder for [2048, 1696] Goppa code

Word to encode:
(0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 

In [14]:
g1.degree()
num_errors

16

In [15]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode to code `y`:")
x2 = D.decode_to_code(y)
print("x = " + str(x2))

print("\nDecode to message `y`:")
word2 = D.decode_to_message(y)
print(word2)


Decode to code `y`:
x = (0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,

In [16]:
x == x2

True

In [17]:
word == word2

True

In [18]:
y + x2 == e

True

In [19]:
def get_weight(c):
    """
    Return the weight of ``c``
    
    INPUT:
        - ``c`` -- a vector
    """
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [20]:
class McEliece:
    r"""
    Implementation of McEliece cryptosystem.
    
    INPUT:
    - ``n`` -- length
    - ``q`` -- Dimension of the finite field `GF(2^q)`
    - ``g``-- a monic polynomial with coefficients in
    a finite field `\GF{2^q}`
    """
    def __init__(self,L, g, K_,S, P):
        """
        Initialize.
        """
        # Goppa code
#         F = GF(2)
#         L = GF(2^q)
#         defining_set = get_defining_set(n, g, L)
        C = Goppa(L, g, K_)
        G = C.get_generator_matrix()
        k = G.nrows()
        n = G.ncols()
        
        
#         # random binary nonsingular matrix
#         S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
#         while rank(S) < k:
#             i = randint(0, k-1)
#             j = randint(0, k-1)
#             S[i,j] = choice(F.list())
        
#         # random permutation matrix
#         columns = list(range(n))
#         P = matrix(F, n)
        
#         for i in range(n):
#             l = randint(0, len(columns)-1)
#             j = columns[l]
#             P[i,j] = 1
#             columns.remove(j)
        self._F = K_
        self._goppa = C
        self._G = G
        self._t = floor(g.degree()/2)
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = self._S  * self._G * self._P
        
    def __repr__(self):
        """
        Representation of a McEliece cryptosystem.
        """
        return "McEliece cryptosystem over {}".format(self._goppa)
        
    def get_S(self):
        """
        Return the random binary nonsingular matrix that is part of the private key
        """
        return self._S
    
    def get_goppa(self):
        """
        Return the C that is part of the private key
        """
        return self._goppa
    
    def get_G(self):
        """
        Return the generating matrix associated with the Goppa code that is part of the private key
        """
        return self._G
    
    def get_P(self):
        """
        Return the random permutation matrix that is part of the private key
        """
        return self._P
    
    def get_public_key(self):
        """
        Return the public key
        """
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m`` -- a plaintext to encrypt
        """
        # random errors
        e = vector([self._F(0) for i in range(self._n)])
        
        while get_weight(e) != self._t:
            i = randint(0, len(e) - 1)
            e[i] = choice(self._F.list())
        
        # chipertext
        c = m * self._public_key + e
        
        return c
    
    def decrypt(self, c):
        """
        Return the plain text associated with the cryptogram ``c``
        
        INPUT:
        - ``c`` -- a chiphertext
        """
        word = c * self._P.transpose()
        D = GoppaDecoder(self._goppa)
        word = D.decode_to_message(word)
        message = word * (self._S).inverse()
        
        return message

In [21]:
k=1696
# random binary nonsingular matrix
S = random_matrix(K_, k)
while (S.determinant()==0):
    S = random_matrix(K_,k)

n=2048
# random permutation matrix
columns = list(range(n))
P = matrix(K_, n)

for i in range(n):
    l = randint(0, len(columns)-1)
    j = columns[l]
    P[i,j] = 1
    columns.remove(j)

In [22]:
show(S)
show(P)

1696 x 1696 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
2048 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [23]:
ME = McEliece(L, g1, K_,S, P)

print("Matriz S:")
show(ME.get_S())

print("Matriz P:")
show(ME.get_P())

print("Matriz G' (clave pública):")
show(ME.get_public_key())

Matriz S:
1696 x 1696 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
Matriz P:
2048 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
Matriz G' (clave pública):
1696 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [24]:
public_key = ME.get_public_key()
message = []
while len(message) != public_key.nrows():
    message = message + [choice(K_.list())]

message = vector(K_, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = ME.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 

In [25]:
message2 = ME.decrypt(encrypted_message)
message2

(0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 

In [26]:
message == message2

True

In [27]:
class Niederreiter :
    r"""
    Implementation of McEliece cryptosystem.
    
    INPUT:
    - ``n`` -- length
    - ``q`` -- Dimension of the finite field `GF(2^q)`
    - ``g``-- a monic polynomial with coefficients in
    a finite field `\GF{2^q}`
    """
    def __init__(self,L,g,K_,S, P):
        """
        Initialize.
        """
        # Goppa code
#         F = GF(2)
#         L = GF(2^q)
#         defining_set = get_defining_set(n, g, L)
        C = Goppa(L, g, K_)
        H = C.get_parity_check_matrix()
        G = C.get_generator_matrix()
        k = G.nrows()
        n = G.ncols()
        
#         # random binary nonsingular matrix
#         S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
#         while rank(S) < k:
#             i = randint(0, k-1)
#             j = randint(0, k-1)
#             S[i,j] = choice(F.list())
        
#         # random permutation matrix
#         columns = list(range(n))
#         P = matrix(F, n)
        
#         for i in range(n):
#             l = randint(0, len(columns)-1)
#             j = columns[l]
#             P[i,j] = 1
#             columns.remove(j)
        
        self._goppa = C
        self._G = G
        self._H = H
        self._t = floor(g.degree()/2)
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._F = K_
        self._public_key = self._S * self._H * self._P
        
    def __repr__(self):
        """
        Representation of a McEliece cryptosystem.
        """
        return "McEliece cryptosystem over {}".format(self._goppa)
        
    def get_S(self):
        """
        Return the random binary nonsingular matrix that is part of the private key
        """
        return self._S
    
    def get_G(self):
        """
        Return the generating matrix associated with the Goppa code that is part of the private key
        """
        return self._G
    
    def get_goppa(self):
        """
        Return the C that is part of the private key
        """
        return self._goppa
    
    def get_H(self):
        """
        Return the parity matrix associated with the Goppa code that is part of the private key
        """
        return self._H
    
    def get_P(self):
        """
        Return the random permutation matrix that is part of the private key
        """
        return self._P
    
    def get_public_key(self):
        """
        Return the public key
        """
        return self._public_key

    def encrypt(self, e):
        """
        Return a chipertext
        
        INPUT:
        - ``m`` -- a plaintext to encrypt
        """
        # random errors
#         e = vector([F(0) for i in range(self._n)])
        
#         while get_weight(e) != self._t:
#             i = randint(0, len(e) - 1)
#             e[i] = choice(F.list())
        
        # chipertext
        c =  e * self._public_key.transpose()
        
        return c
    
    def decrypt(self, c):
        """
        Return the plain text associated with the cryptogram ``c``
        
        INPUT:
        - ``c`` -- a chiphertext
        """
#         y = matrix()

#         show(c * (self._S).inverse().transpose())
        word = (self._H).solve_right(c * (self._S).inverse().transpose())
#         show(word)
#         word = c * self._P.transpose()
        D = GoppaDecoder(self._goppa)
        word_prime = D.decode_to_code(word)
        message = word - word_prime
        message = message * self._P
        
        return message

In [28]:
k=1696
n=2048
# random binary nonsingular matrix
S2 = random_matrix(K_, n-k)
while (S2.determinant()==0):
    S2 = random_matrix(K_,n-k)


# random permutation matrix
columns = list(range(n))
P2 = matrix(K_, n)

for i in range(n):
    l = randint(0, len(columns)-1)
    j = columns[l]
    P2[i,j] = 1
    columns.remove(j)

In [29]:
NIE = Niederreiter(L, g1, K_,S2, P2)

print("Matriz S:")
show(NIE.get_S())

print("Matriz P:")
show(NIE.get_P())

print("Matriz G' (clave pública):")
show(NIE.get_public_key())

Matriz S:
352 x 352 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
Matriz P:
2048 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
Matriz G' (clave pública):
352 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [30]:
# random errors
e = random_error(2048, 16, K_)

In [31]:
print("\nMessage to encrypt:")
print(e)

encrypted_message = NIE.encrypt(e)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [32]:
type(encrypted_message)

sage.modules.vector_mod2_dense.Vector_mod2_dense

In [33]:
show(encrypted_message * (NIE.get_S()).inverse().transpose())

(1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 

In [34]:
# #         show(c * (self._S).inverse().transpose())
#         word = (self._H).solve_right(c * (self._S).inverse().transpose())
#         show(word.str())
# #         word = c * self._P.transpose()
#         D = GoppaDecoder(self._goppa)
#         word_prime = D.decode_to_code(word)
#         message = word - word_prime
#         message = message * self._P

In [35]:
word =NIE.get_H().solve_right(encrypted_message * (NIE.get_S()).inverse().transpose())
show(word)

(0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 

In [36]:
message2 = NIE.decrypt(encrypted_message)
# message2

In [37]:
message2 == e

True

In [38]:
message2 * NIE.get_public_key().transpose() == encrypted_message

True

In [39]:
flag = 0
count = 0
while flag == 0:
    count += 1
    # message_random = random_matrix(K_, n-k,1)
# show(message_random)
    message_random = random_word(n-k, GF(2))
    # message_random = vector(F,message_random)
    # message_random = vector(K_,message_random.list())
    # message_random
    message3 = NIE.decrypt(message_random)
    print(get_weight(message3))
#     show(message_random)
#     show(message3 * NIE.get_public_key().transpose())
    if (message_random ==  message3 * NIE.get_public_key().transpose()):
        flag = 1
        print(get_weight(message3))
        show(message_random)
        show(message3)
        print(count)
    

2
0
0
1
2
1
1
0
0
2
1
1
1
0
1
2
0
0
1
0
1
1
1
2
1
1
1
1
0
0
1
0
1
0
0
2
0
1
0
0
0
1
1
0
1
0
1
0
0
1
2
1
0
3
0
0
2
1
0
2
0
1
1
0
2
3
2
2
4
1
1
2
1
0
0
1
2
0
4
1
1
0
2
1
1
0
1
1
1
1
1
0
2
2
2
1
0
0
1
0
0
2
1
0
2
0
3
1
1
1
0
1
2
4
1
1
1
0
1
0
1
1
3
2
1
1
0
0
1
1
2
2
1
1
0
1
0
1
0
3
2
4
0
0
1
1
0
2
1
0
0
1
0
1
2
1
0
1
1
2
1
0
1
0
1
2
0
0
0
0
0
1
0
1
0
1
0
2
0
1
0
0
3
1
2
0
2
1
2
1
1
1
1
2
1
2
1
2
0
2
1
2
3
3
0
1
1
0
1
3
0
3
0
0
1
1
0
1
0
0
1
1
2
0
1
1
2
0
0
1
0
1
1
0
0
1
2
3
1
0
0
1
1
1
0
0
3
3
0
1
0
2
1
0
2
1
1
2
3
1
0
1
3
1
0
3
1
1
1
2
1
1
0
0
1
0
0
2
0
0
1
2
0
1
1
1
1
0
1
0
0
2
0
0
2
1
0
1
1
1
1
0
1
0
2
1
0
2
3
1
0
1
1
0
0
0
0
1
2
0
0
1
0
0
1
2
0
1
2
1
1
1
2
1
2
1
0
2
1
0
2
0
1
1
0
0
0
2
1
0
2
2
0
4
2
1
0
1
1
1
1
0
0
1
0
2
0
2
0
1
2
1
2
3
2
2
2
1
0
0
0
3
3
1
0
2
0
0
1
1
3
2
1
2
2
0
1
0
1
2
2
1
2
0
0
0
0
2
2
0
0
0
0
0
3
0
0
2
2
1
0
1
2
0
2
1
0
2
0
1
3
2
0
2
0
1
0
1
0
0
1
0
0
4
0
0
3
1
2
1
1
0
2
1
0
1
1
1
0
1
0
0
0
3
3
1
1
3
1
1
0
0
2
0
0
1
1
1
1
0
2
0
0
0
0
1
2
1
0
0
2
0
1
1
3
0
3
0
2
4


KeyboardInterrupt: 

In [ ]:
print(count)

In [53]:
l = 170

In [54]:
# random permutation matrix
columns = list(range(n))
P3 = matrix(K_, n)

for i in range(n):
    tt = randint(0, len(columns)-1)
    j = columns[tt]
    P3[i,j] = 1
    columns.remove(j)

In [55]:
U = matrix(K_, n-k,l)
B = matrix(K_, n-k,n)
A = NIE.get_public_key()
B = A * P3
for j in range(l):
    e_j = random_error(2048, 10, K_)
    c_j = NIE.encrypt(e_j)
    U[:,j] = c_j

In [56]:
show(B)
show(U)

352 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
352 x 170 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [57]:
X_1 = matrix(K_, n,l)
X_2 = matrix(K_, n,l)

In [62]:
for j in range(l):
    flag = 0
    while flag == 0:
        u_j = vector(K_,U[:,j].list())
        x_1j = random_error(2048, 7, K_)
        while get_weight(x_1j) < 7:
            i = randint(0, len(e) - 1)
            x_1j[i] = 1
        v_j = u_j + x_1j * B.transpose()
        x_2j = NIE.decrypt(v_j)
        if (v_j ==  x_2j * NIE.get_public_key().transpose()):
            flag = 1
            print(get_weight(x_2j))
#             show(message_random)
#             show(message3)
            X_1[:,j] = x_1j
            X_2[:,j] = x_2j
    
            

11
14
14
12
9
15
11
10
12
11
12
12
11
10
13
12
13
12
15
14
14
11
11
12
12
10
13
12
12
14
13
12
10
12
9
13
11
14
14
13
12
15
13
9
12
12
11
13
11
14
13
14
11
10
15
10
12
14
13
9
14
14
11
13
13
11
10
15
11
11
13
12
13
12
13
11
11
9
10
12
11
12
14
13
12
11
15
12
13
11
11
14
11
11
10
13
11
10
12
15
10
12
12
12
14
16
11
10
10
11
14
14
12
11
13
13
13
12
13
13
10
12
15
13
14
12
13
11
14
11
11
15
12
8
12
14
11
14
10
9
13
11
12
15
10
9
11
10
12
14
14
12
11
14
9
14
11
13
13
10
10
13
12
16
10
13
12
13
13
9


In [63]:
show(X_1.str())

'[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 